In [80]:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float

from sqlalchemy.ext.declarative import declarative_base
from pprint import pprint
import pandas as pd
import numpy as np

In [40]:
#import data from sql db
Base = declarative_base()

class BaseballPlayer(Base):
    __tablename__ = "player"
    Country = Column(String, primary_key=True)
    year = Column(Integer)
    GDP_Per_capita = Column(Integer)
    education = Column(Integer)
    birth_day = Column(Integer)
    birth_country = Column(String)
    birth_state = Column(String)
    birth_city = Column(String)
    name_first = Column(String)
    name_last = Column(String)
    name_given = Column(String)
    
    height = Column(Integer)
    bats = Column(String)
    throws = Column(String)
    debut = Column(String)
    final_game = Column(String)

C:\Users\Leo\AppData\Local\Temp\ipykernel_39028\1573975637.py:2: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [57]:
Education_df = pd.read_csv("./Clean_datasets/Edu.csv")
G20_pop_df = pd.read_csv("./Clean_datasets/G20.csv")
GDP_per_cap_df = pd.read_csv("./Clean_datasets/GDP.csv")
life_exp_df = pd.read_csv("./Clean_datasets/LifeExp.csv")
unemployment_df = pd.read_csv("./Clean_datasets/Unemployment.csv")
merge6 = pd.read_csv("./Clean_datasets/merge6outer2.csv")


table_df=G20_pop_df.copy()
table_df=table_df.drop(columns=["Unnamed: 0","FREQUENCY","INDICATOR","SUBJECT","MEASURE"])
table_df=table_df.rename(columns={"Value":"Population (MLN)"})
Education_df=Education_df.drop(columns=["Unnamed: 0","FREQUENCY","INDICATOR","MEASURE"])
Education_df=Education_df.rename(columns={"Value":"%_tertiary"})
GDP_per_cap_df=GDP_per_cap_df.drop(columns=["Unnamed: 0","FREQUENCY","INDICATOR","SUBJECT","MEASURE"])
GDP_per_cap_df=GDP_per_cap_df.rename(columns={"Value":"GDP_Per_Capita"})
unemployment_df=unemployment_df.drop(columns=["Unnamed: 0","FREQUENCY","INDICATOR","SUBJECT","MEASURE"])
unemployment_df=unemployment_df.rename(columns={"Value":"Unemployment_%"})
life_exp_df=life_exp_df.drop(columns=["Unnamed: 0","FREQUENCY","INDICATOR","SUBJECT","MEASURE"])
life_exp_df=life_exp_df.rename(columns={"Value":"Life _Expectancy_Birth"})






table_df=pd.merge(table_df,Education_df,how="inner",on=["LOCATION","TIME"])
table_df=pd.merge(table_df,GDP_per_cap_df,how="inner",on=["LOCATION","TIME"])
table_df=pd.merge(table_df,unemployment_df,how="inner",on=["LOCATION","TIME"])
table_df=pd.merge(table_df,life_exp_df,how="inner",on=["LOCATION","TIME"])
table_df

C:\Users\Leo\AppData\Local\Temp\ipykernel_39028\2564662228.py:6: DtypeWarning: Columns (2,3,4,5,8,9,10,11,13,14,15,16,24,26,31,32,33,34,35,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  merge6 = pd.read_csv("./Clean_datasets/merge6outer2.csv")


,LOCATION,TIME,Population (MLN),SUBJECT,%_tertiary,GDP_Per_Capita,Unemployment_%,Life _Expectancy_Birth
0,AUS,1989,16.814416,25_34,34.140335,17835.122957,6.179825,76.4
1,AUS,1991,17.284036,25_34,35.707539,18177.563681,9.576522,77.3
2,AUS,1993,17.634808,25_34,22.836517,20184.946005,10.874670,77.9
3,AUS,1994,17.805468,25_34,23.624947,21432.775454,9.722190,77.9
4,AUS,1995,18.004882,25_34,24.697142,22442.316842,8.472281,77.8
...,...,...,...,...,...,...,...,...
302,USA,2017,325.122128,25_34,47.757000,59589.233923,4.350000,78.6
303,USA,2018,326.838199,25_34,49.373737,62449.612827,3.900000,78.7
304,USA,2019,328.329953,25_34,50.380188,64690.272395,3.666667,78.8
305,USA,2020,331.511512,25_34,51.856071,63480.861222,8.091666,77.0


In [98]:
All_data_df = pd.read_csv("./Clean_datasets/mergefinal.csv")
All_data_df.head()

,LOCATION,Year,G20 Value,Education Value,GDP Value,LifeExp Value,Population Value,TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Unemployment Value
0,Australia,1950,8.1787,NaN,NaN,NaN,8.1787,3.06,24.9,NaN
1,Australia,1951,8.4217,NaN,NaN,NaN,8.4217,3.06,25.1,NaN
2,Australia,1952,8.6365,NaN,NaN,NaN,8.6365,3.18,24.1,NaN
3,Australia,1953,8.8153,NaN,NaN,NaN,8.8153,3.19,23.3,NaN
4,Australia,1954,8.9865,NaN,NaN,NaN,8.9865,3.19,22.3,NaN


In [105]:
#cleaning data by removing all NaN
data_nonulls_df=All_data_df.dropna()
data_nonulls_df=data_nonulls_df.drop(columns="G20 Value")
data_nonulls_df

,LOCATION,Year,Education Value,GDP Value,LifeExp Value,Population Value,TotalFertilityRate(LiveBirthsPerWoman),InfantMortalityRate(InfantDeathsPer1000LiveBirths),Unemployment Value
39,Australia,1989,34.140335,17835.122957,76.4,16.814416,1.84,8.5,6.179825
41,Australia,1991,35.707539,18177.563681,77.3,17.284036,1.86,6.9,9.576522
43,Australia,1993,22.836517,20184.946005,77.9,17.634808,1.87,6.0,10.874670
44,Australia,1994,23.624947,21432.775454,77.9,17.805468,1.85,6.0,9.722190
45,Australia,1995,24.697142,22442.316842,77.8,18.004882,1.83,5.7,8.472281
...,...,...,...,...,...,...,...,...,...
2184,United States of America,2017,47.757000,59589.233923,78.6,325.122128,1.76,5.7,4.350000
2185,United States of America,2018,49.373737,62449.612827,78.7,326.838199,1.72,5.6,3.900000
2186,United States of America,2019,50.380188,64690.272395,78.8,328.329953,1.69,5.5,3.666667
2187,United States of America,2020,51.856071,63480.861222,77.0,331.511512,1.64,5.5,8.091666


In [66]:
#fertility_y = data_nonulls_df['TotalFertilityRate(LiveBirthsPerWoman)']
#data_nonulls_df_x = data_nonulls_df.drop(columns='TotalFertilityRate(LiveBirthsPerWoman)')
#print(fertility_y)

39      1.84
41      1.86
43      1.87
44      1.85
45      1.83
        ... 
2184    1.76
2185    1.72
2186    1.69
2187    1.64
2188    1.66
Name: TotalFertilityRate(LiveBirthsPerWoman), Length: 307, dtype: float64


In [107]:
from sklearn.linear_model import LinearRegression

countries=data_nonulls_df["LOCATION"].unique()

#column_names=data_nonulls_df.columns.values.tolist()
#from sklearn.preprocessing import StandardScaler
#data_nonulls_df = StandardScaler().fit_transform(data_nonulls_df[column_names])

print(countries)



['Australia' 'Canada' 'France' 'Germany' 'Italy' 'Japan'
 'Republic of Korea' 'Mexico' 'Türkiye' 'United Kingdom'
 'United States of America']


In [108]:
regression_per_country={}
for i in range(len(countries)):

    #go through each country and try to do linear regression
    df=data_nonulls_df.loc[(data_nonulls_df["LOCATION"]==countries[i])]
    df.head()
    df=df.drop(columns="LOCATION")

    fertility_y = df['TotalFertilityRate(LiveBirthsPerWoman)']
    df = df.drop(columns='TotalFertilityRate(LiveBirthsPerWoman)')

    data=[]
    model = LinearRegression()
    model.fit(df, fertility_y)
    coef=model.coef_
    intercept=model.intercept_

    data.append(coef)
    data.append(model.intercept_)

    regression_per_country[countries[i]]=data

regression_results = pd.DataFrame(regression_per_country)
regression_results

,Australia,Canada,France,Germany,Italy,Japan,Republic of Korea,Mexico,Türkiye,United Kingdom,United States of America
0,"[-0.2597233793246967, -0.009211851126997789, 6...","[0.014442129295661877, -0.014574918570467464, ...","[0.02111139431291937, 0.002608886672081789, -3...","[0.008277961737760723, 0.0008349701814832121, ...","[0.00021545472410711402, -0.008244141848184514...","[-0.003121839436586233, -0.02374829792018649, ...","[-0.05066081762426732, 0.03030097023285129, -8...","[-0.1785289590862771, -0.0038299991309351912, ...","[-0.3624008481873802, 0.05103751912769172, 1.8...","[-0.08942778092044465, 0.02087983777782906, 2....","[-0.12972215439143528, -0.04877311017493788, 1..."
1,480.874109,-28.294644,-47.948944,-20.304979,-5.179188,11.596216,91.644662,344.644078,721.648854,166.413532,253.627318


In [102]:
regression_per_country

{'Australia': [array([-2.59723379e-01,  2.31990294e-01, -9.21185113e-03,  6.88160601e-06,
          4.17241694e-01,  2.31990294e-01, -1.57918052e-01, -9.27330927e-02]),
  480.8741094269302],
 'Canada': [array([ 1.44421293e-02, -5.30197364e-02, -1.45749186e-02,  2.45067281e-05,
          4.84382115e-02, -5.30197364e-02,  2.91150196e-02,  2.90128072e-02]),
  -28.29464428775533],
 'France': [array([ 2.11113943e-02, -2.48845721e-02,  2.60888667e-03, -3.03242179e-05,
          1.46645818e-01, -2.48845721e-02,  1.95262258e-02, -4.07028027e-02]),
  -47.948944061173385],
 'Germany': [array([ 8.27796174e-03,  1.74381353e-02,  8.34970181e-04, -4.02952397e-06,
          2.89647409e-02,  1.74381353e-02,  5.45802053e-02, -2.19563350e-02]),
  -20.304978989610415],
 'Italy': [array([ 2.15454724e-04,  4.69100442e-02, -8.24414185e-03, -4.49685706e-06,
          1.31782515e-02,  4.69100442e-02,  9.08979155e-03, -2.46037321e-02]),
  -5.17918798542267],
 'Japan': [array([-3.12183944e-03, -1.69328632e-02, 

In [96]:
regression_per_country

{'Australia': 'Australia',
 'Canada': 'Canada',
 'France': 'France',
 'Germany': 'Germany',
 'Italy': 'Italy',
 'Japan': 'Japan',
 'Republic of Korea': 'Republic of Korea',
 'Mexico': 'Mexico',
 'Türkiye': 'Türkiye',
 'United Kingdom': 'United Kingdom',
 'United States of America': 'United States of America'}